In [1]:
import xml.etree.ElementTree as ET
import pandas as pd
import urllib.request

In [2]:
pathFile = '/home/egor/csv_source/data-20150423T0100-structure-20150423T0100.xml'
pathWWW =r'http://data.gov.ru/sites/default/files/opendata/7708234640-threeasevenazeroathreeathree/data-2015-04-23T00-00-00-structure-2015-04-23T00-00-00.xml'

ns={'base': "http://www.SDMX.org/resources/SDMXML/schemas/v1_0/message",
 'common' : "http://www.SDMX.org/resources/SDMXML/schemas/v1_0/common",
 'compact' :"http://www.SDMX.org/resources/SDMXML/schemas/v1_0/compact",
 'cross': "http://www.SDMX.org/resources/SDMXML/schemas/v1_0/cross",
 'generic':"http://www.SDMX.org/resources/SDMXML/schemas/v1_0/generic",
 'query':"http://www.SDMX.org/resources/SDMXML/schemas/v1_0/query",
 'structure':"http://www.SDMX.org/resources/SDMXML/schemas/v1_0/structure",
 'utility':"http://www.SDMX.org/resources/SDMXML/schemas/v1_0/utility",
 'xsi': "http://www.w3.org/2001/XMLSchema-instance",
 'schemaLocation':"http://www.SDMX.org/resources/SDMXML/schemas/v1_0/message SDMXMessage.xsd"}

In [3]:
def from_www(str_path):
    with urllib.request.urlopen(str_path) as response:
        fl_xml=response.read().decode('utf-8')
    return ET.fromstring(fl_xml)

def from_file(str_path):
    tree = ET.parse(str_path)
    return tree.getroot()

root = from_www(pathWWW)

for codelist in root.findall('base:CodeLists', ns):
    codeL_id=codelist.findall('structure:CodeList', ns)
    print (codeL_id[0].attrib['id'])
 

okved


In [37]:
def read_okved():
    okved = root.find('base:CodeLists', ns).find('structure:CodeList[@id="okved"]', ns)
    dct_src=[{'Description':child[0].text, 'okved':int(child.attrib['value'])} 
             for child in okved.findall('structure:Code', ns) ]
    return pd.DataFrame(dct_src)

print (read_okved())

                                          Description   okved
0                                               Всего       0
1                                Ветеринарная деятел.  852001
2                     Деятел. информационных агентств  924001
3              Деятельность в области здравоохранения  851000
4   Деятельность в области радиовещания и телевидения  922000
5                       Деятельность в области спорта  926000
6               Деятельность общественных объединений  910000
7            Деятельность по организации азартных игр  927100
8   Деятельность прочих мест для временного прожив...  552000
9                Деятельность религиозных организаций  913100
10                                        Образование  800000
11                                      Показ фильмов  921300
12                  Предоставление персональных услуг  930001
13           Предоставление прочих персональных услуг  930500
14                    Предоставление социальных услуг  853000
15  Пред

In [32]:
def read_okato():
    okato = root.find('base:CodeLists', ns).find('structure:CodeList[@id="okato"]', ns)
    dct_src=[{'Description':child[0].text, 'okato':int(child.attrib['value'])} 
             for child in okato.findall('structure:Code', ns) ]
    return pd.DataFrame(dct_src)


print (read_okato())

                               Description     okato
0                     Российская Федерация       643
1            Центральный федеральный округ        30
2                     Белгородская область  14000000
3                         Брянская область  15000000
4                     Владимирская область  17000000
5                      Воронежская область  20000000
6                       Ивановская область  24000000
7                        Калужская область  29000000
8                      Костромская область  34000000
9                          Курская область  38000000
10                        Липецкая область  42000000
11                      Московская область  46000000
12                       Орловская область  54000000
13                       Рязанская область  61000000
14                      Смоленская область  66000000
15                      Тамбовская область  68000000
16                        Тверская область  28000000
17                        Тульская область  70

In [53]:
def read_dataset():
    dataset=root.find('base:DataSet', ns)
    
    lstSrc=[{child[0][0].attrib['concept']:int(child[0][0].attrib['value']), 
             child[0][1].attrib['concept']:int(child[0][1].attrib['value']),
             child[1][0].attrib['concept']:child[1][0].attrib['value'],
             child[1][1].attrib['concept']:child[1][1].attrib['value'],
             'Time':int(child[2][0].text), 
             'Value':int(child[2][1].attrib['value'])} for child in dataset]

    return pd.DataFrame(lstSrc, index=range(len(lstSrc)))

print (read_dataset())

               EI                      PERIOD  Time     Value     okato  okved
0   тысяча рублей  значение показателя за год  2011  15178481       643      0
1   тысяча рублей  значение показателя за год  2011   9804662        30      0
2   тысяча рублей  значение показателя за год  2011   1406066  14000000      0
3   тысяча рублей  значение показателя за год  2011      4667  15000000      0
4   тысяча рублей  значение показателя за год  2011     21638  17000000      0
5   тысяча рублей  значение показателя за год  2011     16446  20000000      0
6   тысяча рублей  значение показателя за год  2011     55700  24000000      0
7   тысяча рублей  значение показателя за год  2011     13536  29000000      0
8   тысяча рублей  значение показателя за год  2011       828  34000000      0
9   тысяча рублей  значение показателя за год  2011     25515  38000000      0
10  тысяча рублей  значение показателя за год  2011     11863  42000000      0
11  тысяча рублей  значение показателя за год  2011 

In [47]:
def make_end_dataset():
    df_okved=read_okved().set_index('okved')
    df_res1=pd.merge(read_dataset(), read_okato(), how='inner', on='okato')
    df_res1['okved_name']=df_res1['okved'].map(lambda x:df_okved.loc[x, 'Description'])
    return df_res1

print (make_end_dataset())

               EI                      PERIOD  Time     Value     okato  \
0   тысяча рублей  значение показателя за год  2011  15178481       643   
1   тысяча рублей  значение показателя за год  2011   9804662        30   
2   тысяча рублей  значение показателя за год  2011   1406066  14000000   
3   тысяча рублей  значение показателя за год  2011      4667  15000000   
4   тысяча рублей  значение показателя за год  2011     21638  17000000   
5   тысяча рублей  значение показателя за год  2011     16446  20000000   
6   тысяча рублей  значение показателя за год  2011     55700  24000000   
7   тысяча рублей  значение показателя за год  2011     13536  29000000   
8   тысяча рублей  значение показателя за год  2011       828  34000000   
9   тысяча рублей  значение показателя за год  2011     25515  38000000   
10  тысяча рублей  значение показателя за год  2011     11863  42000000   
11  тысяча рублей  значение показателя за год  2011    484788  46000000   
12  тысяча рублей  значен